# 决策树

决策树由结点（内部结点和叶结点）和有向边构成。

## 特征选择

特征选择在于选取对训练数据具有分类能力的特征。

In [1]:
import numpy as np
import pandas as pd

data = {# 'ID':[i+1 for i in range(15)],
    '年龄':["青年" for i in range(5)]+["中年" for i in range(5)]+["老年" for i in range(5)],
    '有工作':["否","否","是","是","否","否","否","是","否","否","否","否","是","是","否"],
    '有自己的房子':["否","否","否","是","否","否","否","是","是","是","是","是","否","否","否"],
    '信贷情况':["一般","好","好","一般","一般","一般","好","好","非常好","非常好","非常好","好","好","非常好","一般"],
    '类别':["否","否","是","是","否","否","否","是","是","是","是","是","是","是","否"]
}

D = pd.DataFrame(data)
D

,年龄,有工作,有自己的房子,信贷情况,类别
0,青年,否,否,一般,否
1,青年,否,否,好,否
2,青年,是,否,好,是
3,青年,是,是,一般,是
4,青年,否,否,一般,否
5,中年,否,否,一般,否
6,中年,否,否,好,否
7,中年,是,是,好,是
8,中年,否,是,非常好,是
9,中年,否,是,非常好,是


## 信息增益

### 什么是熵？
熵是表示随机变量不确定性的度量。设$X$是一个取有限个数的离散随机变量，其概率分布为$$P(X=x_i)=p_i$$
则随机变量X的熵定义为$$H(p)=H(x)=-\sum^n_{i=1}p_i \ lop \ p_i，其中p_i=0，则定义0 \log0=0$$
熵的单位分别称作bit或者nat。由定义可知，熵只依赖于X的分布，而与X的取值无关。

熵越大，随机变量的不确定性就越大。范围：$0\leq H(P)\leq log \ n $


条件熵$H(Y|X)$表示在已知随机变量$X$的条件下随机变量$Y$的不确定性。随机变量$X$给定的条件下随机变量$Y$的条件熵(conditional entropy)$H(Y|X)$，定义条件下$Y$的条件概率分布的熵对$X$的数学期望$$H(Y|X)=\sum^n_{i=1}p_iH(Y|X=x_i)$$

当熵和条件熵的概率由数据估计(特别是极大似然估计)得到时，所对应的熵与条件熵分别称为经验熵和经验条件熵。

### 什么是信息增益？
信息增益表示得知特征X的信息而使得类Y的信息的不确定性减少的程度。

特征A对训练数据集D的信息增益$g(D,A)$，定义为集合D的经验熵$H(D)$与特征A给定条件下$D$的经验条件熵$H(D|A)$之差，即$$g(D,A)=H(D)-H(D|A)$$
又称为互信息。

根据信息增益准则的特征选择方法是：对训练数据集（或子集）D，计算其每个特征的信息增益，并比较它们的大小，选择信息增益最大的特征。

### 算法步骤
输入：训练数据集$D$和特征$A$
输出：特征$A$对训练数据集D的信息增益$g(D,A)$

（1）计算数据集$D$的经验熵$H(D)$ $$H(D)=-\sum_{k=1}^K\frac{|C_k|}{D}\ log_2\frac{|C_k|}{D}$$
（2）计算特征$A$对数据集$D$的经验条件熵$H(D|A)$ $$H(D|A)=\sum_{i=1}^n\frac{|D_i|}{D}H(D_i)=-\sum_{i=1}^n\frac{|D_{i}|}{D}\sum_{k=1}^K\frac{|D_{ik}|}{D_i}\ log_2\frac{|D_{ik}|}{D_i}$$
（3）计算信息增益熵$$g(D,A)=H(D)-H(D|A)$$

其中，$|D|$为样本容量，$|C_k|$属于类$C_k$的样本个数，$|D_i|$为$A_i$的样本容量，$A_i$表示样本特征，根据特征类别划分$D_i$

In [2]:
D

,年龄,有工作,有自己的房子,信贷情况,类别
0,青年,否,否,一般,否
1,青年,否,否,好,否
2,青年,是,否,好,是
3,青年,是,是,一般,是
4,青年,否,否,一般,否
5,中年,否,否,一般,否
6,中年,否,否,好,否
7,中年,是,是,好,是
8,中年,否,是,非常好,是
9,中年,否,是,非常好,是


In [3]:
def to_number(D):
    D = D.replace("是",1)
    D = D.replace("否",0)
    D = D.replace("青年",0)
    D = D.replace("中年",1)
    D = D.replace("老年",2)
    D = D.replace("一般",0)
    D = D.replace("好",1)
    D = D.replace("非常好",2)
    return D
to_number(D)

,年龄,有工作,有自己的房子,信贷情况,类别
0,0,0,0,0,0
1,0,0,0,1,0
2,0,1,0,1,1
3,0,1,1,0,1
4,0,0,0,0,0
5,1,0,0,0,0
6,1,0,0,1,0
7,1,1,1,1,1
8,1,0,1,2,1
9,1,0,1,2,1


In [4]:
def empirical_entropy(data):
    n_samples = len(data)
    c_unique = list(set(data))
    ci = [list(data[data==c_unique[j]] )for j in range(len(c_unique))]
    return -np.sum([len(di)/n_samples*np.log2(len(di)/n_samples) for di in ci])

def empirical_condaitional_entropy(D,axis=0):
    NewD = D.iloc[:,[axis,-1]]
    nd_unique = list(set(NewD.iloc[:,0]))
    G = 0
    n_samples = len(NewD)
    for i in range(len(nd_unique)):
        newd = NewD[NewD.iloc[:,0] == nd_unique[i]]
        newd_samples = len(newd)
        new_category = newd.iloc[:,-1]
        new_h = empirical_entropy(new_category)
        G += new_h*newd_samples/n_samples
    return G

def information_gain(e,c):
    return e-c

def information_gain_train(D):
    category = D.iloc[:,-1]
    datasets = D.iloc[:,:-1]
    h = empirical_entropy(category)
    n_samples,n_features = D.shape
    G = []
    for i in range(n_features-1):
        c = empirical_condaitional_entropy(D,i)
        g = information_gain(h,c)
        G.append(g)
    max_g = np.max(G)
    print(max_g,D.iloc[:,np.argmax(G)].name)
    return G

G = information_gain_train(D)
G

0.4199730940219748 有自己的房子


[0.08300749985576872,
 0.32365019815155616,
 0.4199730940219748,
 0.36298956253708536]

### 信息增益比

以信息增益作为划分训练数据集的特征，存在偏向于选择取值较多的特征的问题。使用信息增益比可以对这一问题进行校正。

特征$A$对训练数据集$D$的信息增益比$g_R(D,A)$定义为其信息增益$g(D,A)$与训练数据集$D$关于特征$A$的熵$H_A(D)$之比，即$$g_R(D,A)=\frac{g(D,A)}{H_A(D)}$$
其中，$H_A(D)=-\sum_{i=1}^n\frac{|D_i|}{|D|}\ log2\frac{|D_i|}{|D|}$，$n$是特征$A$取值的个数

In [5]:
def infomation_gain_ratio(G,H):
    return G/H

infomation_gain_ratio(G[0],empirical_entropy(D.iloc[:,1]))

0.0903929831514876

## 决策树的生成

### ID3算法

In [6]:
class Node:
    def __init__(self,root=True,label=None,feature_name=None,feature=None,data=None):
        self.root = root
        self.label = label
        self.feature_name = feature_name
        self.feature = feature
        self.tree = {}
        self.data = data
        self.entropy = self.empirical_entropy(self.data.iloc[:,-1])
        self.result = {
#             'label':self.label,
#             'feature':self.feature,
            'entropy':self.entropy,
            'feature_name':self.feature_name,
            'tree':self.tree,
#             'data':self.data
        }
        
    def empirical_entropy(self,data):
        n_samples = len(data)
        c_unique = list(set(data))
        ci = [list(data[data==c_unique[j]] )for j in range(len(c_unique))]
        return -np.sum([len(di)/n_samples*np.log2(len(di)/n_samples) for di in ci])
        
    def add_node(self,val,node):
        self.tree[val] = node
        
    def predict(self,features):
        if self.root is True:
            return self.label
        return self.tree[features[self.feature]].predict(features)
        
    def __repr__(self,):
        return "{}".format(self.result)
    
class DecisionTree:
    def __init__(self,epsilon=0.1):
        self.epsilon = epsilon
        self.tree = {}
        self.alpha = 0.01
        
    # 熵
    @staticmethod
    def empirical_entropy(data):
        n_samples = len(data)
        c_unique = list(set(data))
        ci = [list(data[data==c_unique[j]] )for j in range(len(c_unique))]
        return -np.sum([len(di)/n_samples*np.log2(len(di)/n_samples) for di in ci])
    
    # 条件熵
    @staticmethod
    def empirical_condaitional_entropy(D,axis=0):
        NewD = D.iloc[:,[axis,-1]]
        nd_unique = list(set(NewD.iloc[:,0]))
        G = 0
        n_samples = len(NewD)
        for i in range(len(nd_unique)):
            newd = NewD[NewD.iloc[:,0] == nd_unique[i]]
            newd_samples = len(newd)
            new_category = newd.iloc[:,-1]
            new_h = empirical_entropy(new_category)
            G += new_h*newd_samples/n_samples
        return G
    
    # 信息增益
    @staticmethod
    def information_gain(e,c):
        return e-c
    
    # 获取最优的信息增益
    def information_gain_train(self,D):
        category = D.iloc[:,-1]
        datasets = D.iloc[:,:-1]
        h = empirical_entropy(category)
        n_samples,n_features = D.shape
        G = []
        for i in range(n_features-1):
            c = empirical_condaitional_entropy(D,i)
            g = information_gain(h,c)
            G.append([g,D.iloc[:,i].name])
        best = G[np.argmax(np.array(G)[:,0])]
        return best
    
    def create_tree(self,data):
        
        # 类别
        x_train,y_train = data.iloc[:,:-1],data.iloc[:,-1]
        # 样本特征
        features = data.columns[:-1]
#         print(features)
        
        # 1.若D中所有实例属于同一类Ck，则T为单节点树，并将类Ck作为该节点的类节点，返回T
        # y_train.value_counts() 统计类别以及数量
        # 判断是否只有一个类别
        if len(y_train.value_counts()) == 1:
            return Node(root=True,label=np.unique(y_train).flatten()[0],data=data)
        
        # 2.若A为空，则T为单节点树，并且将D中实例最大的类Ck作为该节点的类标记，返回T
        # A为空，则没有特征可选
        # sort_values：排序
        # index：获取索引下标位置的值
        if len(features) == 0:
            return Node(root=True,label=y_train.value_counts().sort_values(ascending=False).index[0],data=data)
        
        # 3.否则，按算法5.1计算A中各特征对D的信息增益，选择信息增益最大的特征Ag
        max_info_gain,max_feature_name = self.information_gain_train(data)
        
        # 4.如果Ag的信息增益小于阈值epsilon，则置T为单节点树，并将D中实例数最大的类Ck作为节点的类标记，返回T
        if max_info_gain < self.epsilon:
            return Node(root=True,label=y_train.value_counts().sort_values(ascending=False).index[0],data=data)

        # 5.否则，对Ag的每一可能值ai,依Ag=ai将D分割为若干非空子集Di，将Di中实例数最大的类作为标记，
        # 构建子节点，由节点及其子节点构成树T，返回T
        node_tree = Node(root=False,data=data,feature_name=max_feature_name,feature=np.argwhere(np.array(features)==max_feature_name).flatten()[0])
        
        feature_category = data[max_feature_name].value_counts().index
        for fc in feature_category:
            # 划分数据集，根据max_feature_name的类别进行划分，并且删除这一特征的数据
            sub_data = data.loc[data[max_feature_name] == fc].drop([max_feature_name],axis=1)
            # 7.对第i个子节点，以Di为训练集，以A-{Ag}为特征集，递归地调用(1)~(5)，得到子树Ti，返回Ti
            # 递归生成树
            sub_tree = self.create_tree(sub_data)
            node_tree.add_node(fc,sub_tree)

        return node_tree
    
    # 统计叶子结点个数
    def getLeafNumber(self,tree):
        number = 0
        results = tree
        for key in results.keys():
            if type(results[key]).__name__=='dict':
                number += self.getLeafNumber(results[key])
            else:
                number += 1
        return number
    
    # 统计决策树的高度
    def getTreeDepth(self,tree):
        maxdepth = 0
        results = tree.result
        for key in results.keys():
            if type(results[key]).__name__=='dict':
                depth = 1 + self.getLeafNumber(results[key])
                results["entropy"] += depth*self.alpha
            else:
                depth = 1
            maxdepth = depth if depth > maxdepth else maxdepth
        return maxdepth
    
    def pruning(self,tree):
#         x_train,y_train = data.values[:,:-1],data.values[:,-1]
        pass
        
        # 计算每个结点的经验熵
        for i in range(len(x_train)):
            print(self.predict(x_train[i]) == y_train[i])
    
    def fit(self,data):
        self.tree = self.create_tree(data)
#         self.pruning(self.tree,data)
        number = self.getLeafNumber(self.tree.result)
        depth  = self.getTreeDepth(self.tree)
        return self.tree
    
    def predict(self,test):
        return self.tree.predict(test)

data = {
    '年龄':["青年" for i in range(5)]+["中年" for i in range(5)]+["老年" for i in range(5)],
    '有工作':["否","否","是","是","否","否","否","是","否","否","否","否","是","是","否"],
    '有自己的房子':["否","否","否","是","否","否","否","是","是","是","是","是","否","否","否"],
    '信贷情况':["一般","好","好","一般","一般","一般","好","好","非常好","非常好","非常好","好","好","非常好","一般"],
    '类别':["否","否","是","是","否","否","否","是","是","是","是","是","是","是","否"]
}

data = pd.DataFrame(data)


dt = DecisionTree()
dt.fit(data)

{'entropy': 1.0009505944546686, 'feature_name': '有自己的房子', 'tree': {'否': {'entropy': 0.9182958340544896, 'feature_name': '有工作', 'tree': {'否': {'entropy': -0.0, 'feature_name': None, 'tree': {}}, '是': {'entropy': -0.0, 'feature_name': None, 'tree': {}}}}, '是': {'entropy': -0.0, 'feature_name': None, 'tree': {}}}}

In [7]:
dt.predict(['老年', '是', '否', '一般'])

'是'

### sklearn

In [8]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from collections import Counter
import math
from math import log
import pprint

# data
def create_data():
    iris = load_iris()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['label'] = iris.target
    df.columns = [
        'sepal length', 'sepal width', 'petal length', 'petal width', 'label'
    ]
    data = np.array(df.iloc[:100, [0, 1, -1]])
    # print(data)
    return data[:, :2], data[:, -1]


X, y = create_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
import graphviz

In [10]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train,)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [11]:
clf.score(X_test,y_test)

1.0

### C4.5算法

C4.5算对ID3算法进行了改进，C4.5在生成的过程中，用信息增益比来选择特征

## 决策树的剪枝

会出现过拟合。

剪枝：从已生成的树上裁掉一些子树或叶节点，并将其根节点或父节点作为新的叶节点，从而简化分类树的模型。

决策树生成只考虑了通过提高信息增益（或信息增益比）对训练数据进行更好的拟合。而决策树剪枝通过优化损失函数还考虑了减小模型复杂度。决策树生成学习局部的模型，而决策树剪枝学习整体的模型。

### 损失函数
$$C_{\alpha}(T)=\sum^{|T|}_{i=1}N_tH_t(T)+\alpha(T)$$
其中经验熵为$$H_t(T)=-\sum_k\frac{N_{tk}}{N_t}\ log\frac{N_{tk}}{N_t}$$
在损失函数中，右端第一项记作为$$C(T)=\sum_{t=1}^{|T|}N_tH_t(T) = -\sum_{t=1}^{|T|}\sum_{k=1}^{|K|}N_{tk}\ log\frac{N_{tk}}{N_t}$$
这时有，$$C_{\alpha}(T)=C(T)+\alpha|T|$$
其中，$\alpha$为惩罚系数，$T$为树的叶子结点,$|T|$为叶子结点的个数，$H_t(T)$表示第t个叶子的熵，$N_t$表示该叶子所含的训练样例的个数，$C(T)$表示模型对训练数据的预测误差,即模型与训练数据的拟合度。


决策树的剪枝过程（泛化过程）就是从叶子节点开始递归，记其父节点将所有子节点回缩后的子树为$T_b$（分类值取类别比例最大的特征值），未回缩的子树为 $T_a$，如果$C_{\alpha}(Ta)>=C_{\alpha}(Tb)$ 说明回缩后使得损失函数减小了，那么应该使这棵子树回缩，递归直到无法回缩为止，这样使用“贪心”的思想进行剪枝可以降低损失函数值，也使决策树得到泛化。